In [8]:
# imports

import os
import functools
import operator
import copy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
from torchvision import datasets
import torch.optim as optim

import itertools
from itertools import islice
from collections import OrderedDict

from IPython.display import clear_output

Two-layer LSTMs with 20 hidden units in each layer.

Each optimizer is trained using truncated back propagation through time
The minimization is performed using ADAM with a learning rate chosen by random search.

Early stopping when training the optimizer in order to avoid overfitting the optimizer.

After each epoch (some fixed number of learning steps) freeze the optimizer parameters and evaluate its performance.


Trained optimizers are compared with standard optimizers used in Deep Learning: SGD, RMSprop, ADAM, and Nesterov’s accelerated gradient (NAG).

This paper describes a method of NN work optimization with LSTM (long short-term memory).

Optimizer has been applied differently in 4 variants:

1. To optimize random quadratic functions to the other functions from the same distribution.

2. To optimize classic MLP (with different architectures) on MNIST

3. To optimize claasification on CIFAR

4. To implement the optimizer in Neural Art to mix images style



In [9]:
# helpers

def w(v):
    if torch.cuda.is_available():
        return v.cuda()
    return v

def detach_var(v):
    var = w(Variable(v.data, requires_grad=True))
    var.retain_grad()
    return var

def rsetattr(obj, attr, val):
    pre, _, post = attr.rpartition('.')
    return setattr(rgetattr(obj, pre) if pre else obj, post, val)

def rgetattr(obj, attr, *args):
    def _getattr(obj, attr):
        return getattr(obj, attr, *args)
    return functools.reduce(_getattr, [obj] + attr.split('.'))

def to_var(x, requires_grad=True):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, requires_grad=requires_grad)

In [10]:
# loss

# here, we define the target tasks for optimizees

# we build an optimizer which finds vector theta (10-element) 
# theta * W (10x10 matrix) supposed to be as close as possible to y (another 10-elem vector
# from our distribution)

# compute error as Mean Squarred Error

class QuadraticLoss:
    def __init__(self, **kwargs):
        self.W = w(Variable(torch.randn(10, 10)))
        self.y = w(Variable(torch.randn(10)))
        
    def get_loss(self, theta):
        return torch.sum((self.W.matmul(theta) - self.y)**2)

class MNISTLoss: 
    def __init__(self, training=True):
        try:
            os.mkdir('data')
        except:
            pass
        dataset = datasets.MNIST(
            'data', train=True, download=True,
            transform=torchvision.transforms.ToTensor()
        )
        indices = list(range(len(dataset)))
        np.random.RandomState(10).shuffle(indices)
        if training:
            indices = indices[:len(indices) // 2]
        else:
            indices = indices[len(indices) // 2:]

        # sample MNIST dataset to batches with 128 size    
        self.loader = torch.utils.data.DataLoader(
            dataset, batch_size=128,
            sampler=torch.utils.data.sampler.SubsetRandomSampler(indices))

        self.batches = []
        self.cur_batch = 0
        
    def sample(self):
        if self.cur_batch >= len(self.batches):
            self.batches = []
            self.cur_batch = 0
            for b in self.loader:
                self.batches.append(b)
        batch = self.batches[self.cur_batch]
        self.cur_batch += 1
        return batch

In [11]:
# meta module
class MetaModule(nn.Module):
    def parameters(self):
       for name, param in self.named_params(self):
            yield param

    def named_parameters(self):
       for name, param in self.named_params(self):
            yield name, param
    
    def named_leaves(self):
        return []
    
    def named_submodules(self):
        return []
    
    def named_params(self, curr_module=None, memo=None, prefix=''):       
        if memo is None:
            memo = set()

        if hasattr(curr_module, 'named_leaves'):
            for name, p in curr_module.named_leaves():
                if p is not None and p not in memo:
                    memo.add(p)
                    yield prefix + ('.' if prefix else '') + name, p
        else:
            for name, p in curr_module._parameters.items():
                if p is not None and p not in memo:
                    memo.add(p)
                    yield prefix + ('.' if prefix else '') + name, p
                    
        for mname, module in curr_module.named_children():
            submodule_prefix = prefix + ('.' if prefix else '') + mname
            for name, p in self.named_params(module, memo, submodule_prefix):
                yield name, p
    
    def update_params(self, lr_inner, first_order=False, source_params=None, detach=False):
        if source_params is not None:
            for tgt, src in zip(self.named_params(self), source_params):
                name_t, param_t = tgt
                grad = src
                if first_order:
                    grad = to_var(grad.detach().data)
                tmp = param_t - lr_inner * grad
                self.set_param(self, name_t, tmp)
        else:
            for name, param in self.named_params(self):
                if not detach:
                    grad = param.grad
                    if first_order:
                        grad = to_var(grad.detach().data)
                    tmp = param - lr_inner * grad
                    self.set_param(self, name, tmp)
                else:
                    param = param.detach_()
                    self.set_param(self, name, param)

    def set_param(self,curr_mod, name, param):
        if '.' in name:
            n = name.split('.')
            module_name = n[0]
            rest = '.'.join(n[1:])
            for name, mod in curr_mod.named_children():
                if module_name == name:
                    self.set_param(mod, rest, param)
                    break
        else:
            setattr(curr_mod, name, param)
            
    def detach_params(self):
        for name, param in self.named_params(self):
            self.set_param(self, name, param.detach())   
                
    def copy(self, other, same_var=False):
        for name, param in other.named_params():
            if not same_var:
                param = to_var(param.data.clone(), requires_grad=True)
            self.set_param(name, param)


class MetaLinear(MetaModule):
    def __init__(self, *args, **kwargs):
        super().__init__()
        ignore = nn.Linear(*args, **kwargs)
       
        self.register_buffer('weight', to_var(ignore.weight.data, requires_grad=True))
        self.register_buffer('bias', to_var(ignore.bias.data, requires_grad=True))
        self.in_features = ignore.weight.size(1)
        self.out_features = ignore.weight.size(0)
        
    def forward(self, x):
        return F.linear(x, self.weight, self.bias)
    
    def named_leaves(self):
        return [('weight', self.weight), ('bias', self.bias)]

In [12]:
# meta optimizer
# here the optimizer of the network is built

class Optimizer_LSTM(nn.Module):
    def __init__(self, preproc=False, hidden_sz=20, preproc_factor=10.0):
        super().__init__()
        self.hidden_sz = hidden_sz
        if preproc:
            self.recurs = nn.LSTMCell(2, hidden_sz)
        else:
            self.recurs = nn.LSTMCell(1, hidden_sz)
        self.recurs2 = nn.LSTMCell(hidden_sz, hidden_sz)
        self.output = nn.Linear(hidden_sz, 1)
        self.preproc = preproc
        self.preproc_factor = preproc_factor
        self.preproc_threshold = np.exp(-preproc_factor)
        
    def forward(self, inp, hidden, cell):
        if self.preproc:
            inp = inp.data
            inp2 = w(torch.zeros(inp.size()[0], 2))
            keep_grads = (torch.abs(inp) >= self.preproc_threshold).squeeze()
            inp2[:, 0][keep_grads] = (torch.log(torch.abs(inp[keep_grads]) + 1e-8) / self.preproc_factor).squeeze()
            inp2[:, 1][keep_grads] = torch.sign(inp[keep_grads]).squeeze()
            
            inp2[:, 0][~keep_grads] = -1
            inp2[:, 1][~keep_grads] = (float(np.exp(self.preproc_factor)) * inp[~keep_grads]).squeeze()
            inp = w(Variable(inp2))
        hidden0, cell0 = self.recurs(inp, (hidden[0], cell[0]))
        hidden1, cell1 = self.recurs2(hidden0, (hidden[1], cell[1]))
        return self.output(hidden1), (hidden0, hidden1), (cell0, cell1)

In [13]:
# optimizee
# here the optimizers for the tasks (for the experiments with quadratic
# functions and MNIST dataset) are built

class QuadOptimizee(MetaModule):
    def __init__(self, theta=None):
        super().__init__()
        self.register_buffer('theta', to_var(torch.zeros(10).cuda(), requires_grad=True))
        
    def forward(self, target):
        return target.get_loss(self.theta)
    
    def all_named_parameters(self):
        return [('theta', self.theta)]

class QuadOptimizeeNormal(nn.Module):
    def __init__(self, theta=None):
        super().__init__()
        if theta is None:
            self.theta = nn.Parameter(torch.zeros(10))
        else:
            self.theta = theta
        
    def forward(self, target):
        return target.get_loss(self.theta)
    
    def all_named_parameters(self):
        return [('theta', self.theta)]

# optimizee for MNIST is a neural network with 1 layer and 20 hidden units
# parameter of this function is activation which is used to change
# the activation function as it has been done in the paper

def create_MNISTNet(activation):
    class MNISTNet(MetaModule):
        def __init__(self, layer_size=20, n_layers=1, **kwargs):
            super().__init__()

            inp_size = 28*28
            self.layers = {}
            for i in range(n_layers):
                self.layers[f'mat_{i}'] = MetaLinear(inp_size, layer_size)
                inp_size = layer_size

            self.layers['final_mat'] = MetaLinear(inp_size, 10)
            self.layers = nn.ModuleDict(self.layers)

            self.activation = activation
            self.loss = nn.NLLLoss()

        def all_named_parameters(self):
            return [(k, v) for k, v in self.named_parameters()]

        def forward(self, loss):
            inp, out = loss.sample()
            inp = w(Variable(inp.view(inp.size()[0], 28*28)))
            out = w(Variable(out))

            cur_layer = 0
            while f'mat_{cur_layer}' in self.layers:
                inp = self.activation(self.layers[f'mat_{cur_layer}'](inp))
                cur_layer += 1

            inp = F.log_softmax(self.layers['final_mat'](inp), dim=1)
            l = self.loss(inp, out)
            return l
    return MNISTNet

In [14]:
# training

# here the one epoch training of the optimizer is built

def one_step_fit_LSTM(opt_net, meta_opt, target_cls, target_to_opt, \
                      unroll, optim_it, out_mul, should_train=True):
    """
    Parameters:
        - opt_net: 
            an instance of Optimizer_LSTM class
        - meta_opt:
            a normal optimizer, e.g. Adam
        - target_cls: 
            loss function, an instance of QuadraticLoss, or MNISTLoss, or other similar classes.
        - target_to_opt:
            an optimizee, an instance of QuadOptimizee, or MNISTNet, or other similar classes.
        - unroll:
            int, how often make a gradient step for LSTM-based optimizer (measuring in training steps of optimizee)
        - optim_it:
            int, how many steps train an optimizee
        - out_mul:
            float, a multiplier of the LSTM-based optimizer outputs (usual learning rate in some sense)
        - should_train: 
            bool, whether the LSTM-based optimizer is trained or not.
    Returns:
        - all_losses_ever: 
            list, all losses of the optimizee for optim_it steps.
    """
    if should_train:
        opt_net.train()
    else:
        opt_net.eval()
        unroll = 1
    
    # Initialization
    target = target_cls(training=should_train)
    optimizee = w(target_to_opt())
    n_params = 0
    for name, p in optimizee.all_named_parameters():
        n_params += int(np.prod(p.size()))
    # Hidden and cell states for LSTM
    hidden_states = [w(Variable(torch.zeros(n_params, opt_net.hidden_sz))) for _ in range(2)]
    cell_states = [w(Variable(torch.zeros(n_params, opt_net.hidden_sz))) for _ in range(2)]
    all_losses_ever = []
    if should_train:
        meta_opt.zero_grad()
    all_losses = None
    # Usual training
    for iteration in range(1, optim_it + 1):
        loss = optimizee(target)
                    
        if all_losses is None:
            all_losses = loss
        else:
            all_losses += loss
        
        all_losses_ever.append(loss.data.cpu().numpy())
        loss.backward(retain_graph=should_train)
        
        # LSTM step
        offset = 0
        result_params = {}
        hidden_states2 = [w(Variable(torch.zeros(n_params, opt_net.hidden_sz))) for _ in range(2)]
        cell_states2 = [w(Variable(torch.zeros(n_params, opt_net.hidden_sz))) for _ in range(2)]
        for name, p in optimizee.all_named_parameters():
            cur_sz = int(np.prod(p.size()))
            # We do this so the gradients are disconnected from the graph but we still get
            # gradients from the rest
            gradients = detach_var(p.grad.view(cur_sz, 1))
            updates, new_hidden, new_cell = opt_net(
                gradients,
                [h[offset:offset+cur_sz] for h in hidden_states],
                [c[offset:offset+cur_sz] for c in cell_states]
            )
            for i in range(len(new_hidden)):
                hidden_states2[i][offset:offset+cur_sz] = new_hidden[i]
                cell_states2[i][offset:offset+cur_sz] = new_cell[i]
            result_params[name] = p + updates.view(*p.size()) * out_mul
            result_params[name].retain_grad()
            
            offset += cur_sz
        # Optimization of LSTM     
        if iteration % unroll == 0:
            if should_train:
                meta_opt.zero_grad()
                all_losses.backward()
                meta_opt.step()
                
            all_losses = None

            optimizee = w(target_to_opt())
            optimizee.load_state_dict(result_params)
            optimizee.zero_grad()
            hidden_states = [detach_var(v) for v in hidden_states2]
            cell_states = [detach_var(v) for v in cell_states2]
            
        else:
            for name, p in optimizee.all_named_parameters():
                rsetattr(optimizee, name, result_params[name])
            assert len(list(optimizee.all_named_parameters()))
            hidden_states = hidden_states2
            cell_states = cell_states2
    return all_losses_ever

def fit(target_cls, target_to_opt, model_name, preproc=False, unroll=20, \
        optim_it=100, n_epochs=20, n_tests=100, lr=0.001, out_mul=1.0, norm=True):
    """
    This function is used to train the model-based optimizers
    Parameters:
        - target_cls: 
            loss function, an instance of QuadraticLoss, or MNISTLoss, or other similar classes.
        - target_to_opt:
            an optimizee, an instance of QuadOptimizee, or MNISTNet, or other similar classes.
        - model_name:
            str, 'LSTM' or 'HNN' only
        - preproc:
            bool, whether to use preprocessing in model-based optimizers or not. 
            Usually, it is used with neural networks optimizees, details in meta_optimizer.py
        - unroll:
            int, how often make a gradient step for model-based optimizer (measuring in training steps of optimizee)
        - optim_it:
            int, how many steps train in one epoch
        - n_epochs:
            int, number of epochs to train optimizers
        - n_tests:
            int, how many test steps to do after one epoch of training
        - lr:
            float, learning rate of the optimizers that train model-based optimizers
        - out_mul:
            float, a multiplier of the model-based optimizer outputs (usual learning rate in some sense)
    Returns:
        - best_loss: 
            float, the best test loss
        - best_net:
            an instance of Optimizer_LSTM or Optimizer_HNN classes which corresponds to best_loss
    """
    opt_net = w(Optimizer_LSTM(preproc=preproc))     
    meta_opt = optim.Adam(opt_net.parameters(), lr=lr)
    best_net = None
    best_loss = np.inf
    
    for _ in range(n_epochs):
        for _ in range(20):
          one_step_fit_LSTM(opt_net, meta_opt, target_cls, target_to_opt, \
                            unroll, optim_it, out_mul, should_train=True)
          loss = np.mean([np.mean(one_step_fit_LSTM(opt_net, meta_opt, target_cls, target_to_opt, \
                                                    unroll, optim_it, out_mul, should_train=False)) for _ in range(n_tests)])
        if loss < best_loss:
            best_loss = loss
            best_net = copy.deepcopy(opt_net.state_dict())
    return best_loss, best_net

def fit_normal(target_cls, target_to_opt, opt_class, lr, n_tests=100, n_epochs=100, **kwargs):
    """
    This function is used to train optimizees by normal optimizers
    Parameters:
        - target_cls: 
            loss function, an instance of QuadraticLoss, or MNISTLoss, or other similar classes.
        - target_to_opt:
            an optimizee, an instance of QuadOptimizee, or MNISTNet, or other similar classes.
        - opt_class:
            an optimizer, for example Adam or SGD (from torch.optim)
        - n_epochs:
            int, number of epochs to train optimizers
        - n_tests:
            int, how many test steps to do after one epoch of training
    Returns:
        - results: 
            list, a list of all loss values
    """
    results = []
    for i in range(n_tests):
        target = target_cls(training=False)
        optimizee = w(target_to_opt())
        optimizer = opt_class(optimizee.parameters(), lr=lr, **kwargs)
        total_loss = []
        for _ in range(n_epochs):
            loss = optimizee(target)
            
            total_loss.append(loss.data.cpu().numpy())
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        results.append(total_loss)
    return results

In [15]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

sns.set(color_codes=True)
sns.set_style("white")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Quadratic functions
We build an optimizee which finds vector theta (10-element) theta * W (10x10 matrix) supposed to be as close as possible to y (another 10-elem vector from our distribution).

Then, we apply the optimizer to this quadratic optimizee.

In [ ]:
loss_LSTM, quad_optimizer_LSTM = fit(QuadraticLoss, QuadOptimizee, 'LSTM', unroll=20, optim_it=100, lr=0.003,\
                                          n_tests=50, n_epochs=100)

In [ ]:
print('Best loss of LSTM = ', loss_LSTM)

In [ ]:
fit_data = np.zeros((100, 100, 6))

opt = w(Optimizer_LSTM())
opt.load_state_dict(quad_optimizer_LSTM)
fit_data[:, :, 0] = np.array([one_step_fit_LSTM(opt, None, QuadraticLoss, QuadOptimizee, \
                                                1, 100, out_mul=1.0, should_train=False) for _ in range(100)])
QUAD_LRS = [0.1, 0.03, 0.01, 0.01]
NORMAL_OPTS = [(optim.Adam, {}), (optim.RMSprop, {}), (optim.SGD, {'momentum': 0.9}), (optim.SGD, {'nesterov': True, 'momentum': 0.9})]
for i, ((opt, extra_kwargs), lr) in enumerate(zip(NORMAL_OPTS, QUAD_LRS)):
    fit_data[:, :, 1 + i] = np.array(fit_normal(QuadraticLoss, QuadOptimizeeNormal, opt, lr=lr, **extra_kwargs))

In [ ]:
np.save('figure_one', fit_data)

In [ ]:
names = ['LSTM', 'ADAM', 'RMSprop', 'SGD', 'NAG']
for i, name in enumerate(names):
    print('{}: {:.2f}'.format(name, np.mean(fit_data, axis=0)[-1, i]))

In [ ]:
plt.figure(figsize=(8,6))
for i, opt in enumerate(names):
    plt.plot(np.mean(fit_data[:,:,i], axis=0), label=opt)
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.yscale('log')
plt.title('Quadratic functions')
plt.legend()
plt.grid()
plt.savefig('quad.pdf')
plt.show()

# 2. MNIST dataset

Again, we implement the optimizer to the new optimizee.
MNIST optimizee has a single hidden layer with 20 hidden units network.

Originally, the optimizer was trained with the sigmoid function and then we apply ReLU activation function to check how the optimizer works in this case.

In [ ]:
loss_LSTM, MNIST_optimizer_LSTM = fit(MNISTLoss, create_MNISTNet(nn.Sigmoid()), 'LSTM', unroll=20, optim_it=100, lr=0.01,\
                                           out_mul=0.1, preproc=True, n_tests=20, n_epochs=50)

In [ ]:
print('Best loss of LSTM = ', loss_LSTM)

Sigmoid activation function

In [ ]:
fit_data = np.zeros((100, 100, 6))

opt = w(Optimizer_LSTM(preproc=True))


opt.load_state_dict(MNIST_optimizer_LSTM)
fit_data[:, :, 0] = np.array([one_step_fit_LSTM(opt, None, MNISTLoss, create_MNISTNet(nn.Sigmoid()), \
                                                1, 100, out_mul=1.0, should_train=False) for _ in range(100)])
QUAD_LRS = [0.03, 0.01, 1.0, 1.0]
NORMAL_OPTS = [(optim.Adam, {}), (optim.RMSprop, {}), (optim.SGD, {'momentum': 0.9}), (optim.SGD, {'nesterov': True, 'momentum': 0.9})]
for i, ((opt, extra_kwargs), lr) in enumerate(zip(NORMAL_OPTS, QUAD_LRS)):
    fit_data[:, :, 2 + i] = np.array(fit_normal(MNISTLoss, create_MNISTNet(nn.Sigmoid()), opt, lr=lr, **extra_kwargs))

In [ ]:
np.save('mnist_fit_data_sigmoid', fit_data)

In [ ]:
names = ['LSTM', 'ADAM', 'RMSprop', 'SGD', 'NAG']
for i, name in enumerate(names):
    print('{}: {:.2f}'.format(name, np.mean(fit_data, axis=0)[-1, i]))

In [ ]:
plt.figure(figsize=(8,6))
for i, opt in enumerate(names):
    plt.plot(np.mean(fit_data[:,:,i], axis=0), label=opt)
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('MNIST, Sigmoid')
plt.legend()
plt.yscale('log')
plt.grid()
plt.savefig('mnist_sigmoid.pdf')
plt.show()

ReLU activation function

In [ ]:
fit_data = np.zeros((100, 100, 6))

opt = w(Optimizer_LSTM(preproc=True))
opt.load_state_dict(MNIST_optimizer_LSTM)
fit_data[:, :, 0] = np.array([one_step_fit_LSTM(opt, None, MNISTLoss, create_MNISTNet(nn.ReLU()), \

QUAD_LRS = [0.03, 0.003, 0.3, 0.3]
NORMAL_OPTS = [(optim.Adam, {}), (optim.RMSprop, {}), (optim.SGD, {'momentum': 0.9}), (optim.SGD, {'nesterov': True, 'momentum': 0.9})]
for i, ((opt, extra_kwargs), lr) in enumerate(zip(NORMAL_OPTS, QUAD_LRS)):
    fit_data[:, :, 2 + i] = np.array(fit_normal(MNISTLoss, create_MNISTNet(nn.ReLU()), opt, lr=lr, **extra_kwargs))

In [ ]:
np.save('mnist_fit_data_relu', fit_data)

In [ ]:
names = ['LSTM', 'ADAM', 'RMSprop', 'SGD', 'NAG']
for i, name in enumerate(names):
    print('{}: {:.2f}'.format(name, np.mean(fit_data, axis=0)[-1, i]))

In [ ]:
plt.figure(figsize=(8,6))
for i, opt in enumerate(names):
    plt.plot(np.mean(fit_data[:,:,i], axis=0), label=opt)
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('MNIST, ReLU')
plt.legend()
plt.yscale('log')
plt.grid()
plt.savefig('mnist_relu.pdf')
plt.show()